In [1]:
import pathlib
import random
from decouple import config
from replicate.client import Client

In [2]:
NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent
DATA_DIR = REPO_DIR / "data"
GENERATED_DIR = DATA_DIR / "generated"
GENERATED_DIR.mkdir(exist_ok=True, parents=True)
REPLICATE_API_TOKEN = config("REPLICATE_API_TOKEN")
REPLICATE_MODEL = config("REPLICATE_MODEL", default="enricd/superme-enricd-v1")
REPLICATE_MODEL_VERSION = config("REPLICATE_MODEL_VERSION", default="50ed285c7114c60eb8a564084ae68f813134144377caf86741e3d1a9de90131d")

replicate_client = Client(api_token=REPLICATE_API_TOKEN)

In [ ]:
model = f"{REPLICATE_MODEL}:{REPLICATE_MODEL_VERSION}"
prompt = "a photo of TOK adult man dressed up for a professional photo shoot"

responses = replicate_client.run(
    model,
    input={
        "model": "dev",
        "prompt": prompt,
        "go_fast": False,
        # "lora_scale": 1,
        "megapixels": "1",
        "num_outputs": 2,
        # "aspect_ratio": "1:1",
        "output_format": "jpg",
        # "guidance_scale": 3,
        # "output_quality": 80,
        # "prompt_strength": 0.8,
        # "extra_lora_scale": 1,
        # "num_inference_steps": 28
    }
)
print(responses)

[<replicate.helpers.FileOutput object at 0x00000264517D74D0>, <replicate.helpers.FileOutput object at 0x000002645178F110>]


In [ ]:
session_id = random.randint(10000, 99999)
for i, output in enumerate(responses):
    fname = f"{session_id}-{i}.jpg"
    outpath = GENERATED_DIR / fname
    with open(outpath, "wb") as f:
        f.write(output.read())

    print(i, output)

0 https://replicate.delivery/xezq/d3KHerIbVASgbCOCr9iFFFAtVufIlIjLWusQ1Y5SvAh3JDQUA/out-0.jpg
1 https://replicate.delivery/xezq/WAefjIC3W5n1lE1Ni9kQAdmDFIeepNDJDJcWc10Zrq8eOZAiC/out-1.jpg


In [5]:
rep_model = replicate_client.models.get(REPLICATE_MODEL)
rep_version = rep_model.versions.get(REPLICATE_MODEL_VERSION) 

prompt = "a photo of TOK adult man dressed up for a professional photo shoot"
input_args = {
    "prompt": prompt,
    "num_outputs": 2,
    "output_format": "jpg",
}

pred = replicate_client.predictions.create(
    version=rep_version,
    input=input_args,
)

In [6]:
pred.id

'6wz4x6q0p5rma0cn254rv2dzxc'

In [8]:
pred.status

'starting'

In [9]:
pred_id = pred.id
pred_lookup = replicate_client.predictions.get(pred_id)

In [10]:
pred_lookup.status

'succeeded'

In [14]:
pred_urls = pred_lookup.output
pred_urls

['https://replicate.delivery/xezq/3ffATdVGxFpirEmh704LinkKdnmk0DH7MesnA7e5bHHA5MARB/out-0.jpg',
 'https://replicate.delivery/xezq/QTzi2VehpSxxaKU4JaxZYJyPmh4yWEW4engPcmIfOoXhcGgoA/out-1.jpg']

In [ ]:
import httpx
import random

session_id = random.randint(10000, 99999)
with httpx.Client() as client:
    for i, url in enumerate(pred_urls):
        fname = f"{session_id}-{i}.jpg"
        outpath = GENERATED_DIR / fname
        res = client.get(url)
        res.raise_for_status()
        with open(outpath, "wb") as f:
            f.write(res.content)

        print(i, output)

0 https://replicate.delivery/xezq/WAefjIC3W5n1lE1Ni9kQAdmDFIeepNDJDJcWc10Zrq8eOZAiC/out-1.jpg
1 https://replicate.delivery/xezq/WAefjIC3W5n1lE1Ni9kQAdmDFIeepNDJDJcWc10Zrq8eOZAiC/out-1.jpg


: 